In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Alzheimers_Disease"
cohort = "GSE122063"

# Input paths
in_trait_dir = "../../input/GEO/Alzheimers_Disease"
in_cohort_dir = "../../input/GEO/Alzheimers_Disease/GSE122063"

# Output paths
out_data_file = "../../output/preprocess/Alzheimers_Disease/GSE122063.csv"
out_gene_data_file = "../../output/preprocess/Alzheimers_Disease/gene_data/GSE122063.csv"
out_clinical_data_file = "../../output/preprocess/Alzheimers_Disease/clinical_data/GSE122063.csv"
json_path = "../../output/preprocess/Alzheimers_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Dementia Comparison: VaD vs. AD vs. Controls"
!Series_summary	"Gene expression profiling was performed on frontal and temporal cortex from vascular dementia (VaD), Alzheimer's disease (AD), and non-demented controls (Control) obtained from the University of Michigan Brain Bank. Controls and AD cases had no infarcts in the autopsied hemisphere. Vascular dementia cases had low Braak staging."
!Series_overall_design	"Each sample (VaD=8), (AD=12), (Controls=11) was run, at a minimum, in duplicate with a dye swap (Cy3/Cy5) on Agilent Human 8x60k v2 microarrays."
!Series_overall_design	""
!Series_overall_design	"These are dual channel arrays, but have been processed as a single channel analysis. Normalized log2 signal is provided for each sample.  Raw files are included in a tar archive on the series record. Please see 'Description' field for the name of the raw file for each sample."
Sample Characteristics Dictionary:
{0: ['patient diagnosis: Vascular 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the Series_title, Series_summary, and Series_overall_design, this dataset contains gene expression data
# from frontal and temporal cortex samples using Agilent Human 8x60k v2 microarrays.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Trait (Alzheimer's Disease) - available in key 0 "patient diagnosis"
trait_row = 0

# Age - available in key 6 "age"
age_row = 6

# Gender - available in key 5 "Sex"
gender_row = 5

# 2.2 Data Type Conversion

def convert_trait(val):
    """Convert trait values to binary (0: Control, 1: Alzheimer's disease)"""
    if not isinstance(val, str):
        return None
    
    if ":" in val:
        val = val.split(":", 1)[1].strip()
    
    if "Alzheimer's disease" in val or "AD" in val:
        return 1
    elif "Control" in val:
        return 0
    else:  # "Vascular dementia" or other values
        return None  # We only want AD vs Control

def convert_age(val):
    """Convert age values to continuous numeric values"""
    if not isinstance(val, str):
        return None
    
    if ":" in val:
        val = val.split(":", 1)[1].strip()
    
    try:
        return float(val)
    except:
        return None

def convert_gender(val):
    """Convert gender values to binary (0: Female, 1: Male)"""
    if not isinstance(val, str):
        return None
    
    if ":" in val:
        val = val.split(":", 1)[1].strip().lower()
    else:
        val = val.lower()
    
    if "female" in val:
        return 0
    elif "male" in val:
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Save initial filtering information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    preview = preview_df(clinical_df)
    print("Clinical Data Preview:")
    print(preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Data Preview:
{'GSM3454053': [nan, 75.0, 1.0], 'GSM3454054': [nan, 75.0, 1.0], 'GSM3454055': [nan, 75.0, 1.0], 'GSM3454056': [nan, 75.0, 1.0], 'GSM3454057': [nan, 75.0, 1.0], 'GSM3454058': [nan, 75.0, 1.0], 'GSM3454059': [nan, 75.0, 1.0], 'GSM3454060': [nan, 75.0, 1.0], 'GSM3454061': [nan, 90.0, 0.0], 'GSM3454062': [nan, 90.0, 0.0], 'GSM3454063': [nan, 90.0, 0.0], 'GSM3454064': [nan, 90.0, 0.0], 'GSM3454065': [nan, 78.0, 1.0], 'GSM3454066': [nan, 78.0, 1.0], 'GSM3454067': [nan, 78.0, 1.0], 'GSM3454068': [nan, 78.0, 1.0], 'GSM3454069': [nan, 82.0, 0.0], 'GSM3454070': [nan, 82.0, 0.0], 'GSM3454071': [nan, 82.0, 0.0], 'GSM3454072': [nan, 82.0, 0.0], 'GSM3454073': [nan, 96.0, 0.0], 'GSM3454074': [nan, 96.0, 0.0], 'GSM3454075': [nan, 96.0, 0.0], 'GSM3454076': [nan, 96.0, 0.0], 'GSM3454077': [nan, 77.0, 1.0], 'GSM3454078': [nan, 77.0, 1.0], 'GSM3454079': [nan, 77.0, 1.0], 'GSM3454080': [nan, 77.0, 1.0], 'GSM3454081': [nan, 93.0, 0.0], 'GSM3454082': [nan, 93.0, 0.0], 'GSM3454083': [n

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16',
       '17', '18', '19', '20', '21', '22', '23'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# This is not a code execution step but an assessment of gene identifiers
# Looking at the provided indices which are numeric values like '4', '5', '6', etc.
# These are not standard human gene symbols (which would be alphanumeric like 'APOE', 'PSEN1', etc.)
# These appear to be probe IDs or some other numerical identifiers that would need mapping to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['192', '192', '192', '192', '192'], 'ROW': [328.0, 326.0, 324.0, 322.0, 320.0], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'A_23_P117082', 'A_33_P3246448'], 'SPOT_ID': ['CONTROL', 'CONTROL', 'CONTROL', 'A_23_P117082', 'A_33_P3246448'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_015987', 'NM_080671'], 'GB_ACC': [nan, nan, nan, 'NM_015987', 'NM_080671'], 'LOCUSLINK_ID': [nan, nan, nan, 50865.0, 23704.0], 'GENE_SYMBOL': [nan, nan, nan, 'HEBP1', 'KCNE4'], 'GENE_NAME': [nan, nan, nan, 'heme binding protein 1', 'potassium voltage-gated channel, Isk-related family, member 4'], 'UNIGENE_ID': [nan, nan, nan, 'Hs.642618', 'Hs.348522'], 'ENSEMBL_ID': [nan, nan, nan, 'ENST00000014930', 'ENST00000281830'], 'ACCESSION_STRING': [nan, nan, nan, 'ref|NM_015987|ens|ENST00000014930|gb|AF117615|gb|BC016277', 'ref|NM_080671|ens|ENST00000281830|tc|THC2655788'], 'CHROMOSOMAL_LOCATION': 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Observe the gene identifiers and decide on the mapping columns
# From the preview, we can see that 'ID' in gene_annotation corresponds to row identifiers in gene_data
# And 'GENE_SYMBOL' contains the human gene symbols we need

# 2. Get a gene mapping dataframe with the ID and GENE_SYMBOL columns
mapping_data = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')
print(f"Mapping data shape: {mapping_data.shape}")
print("First 5 rows of mapping data:")
print(mapping_data.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(expression_df=gene_data, mapping_df=mapping_data)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First 5 gene symbols after mapping:")
print(gene_data.index[:5])


Mapping data shape: (54295, 2)
First 5 rows of mapping data:
  ID          Gene
3  4         HEBP1
4  5         KCNE4
5  6        BPIFA3
6  7  LOC100129869
7  8          IRG1


Gene expression data shape after mapping: (20353, 136)
First 5 gene symbols after mapping:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M'], dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
print("Loading the original clinical data...")
# Get the matrix file again to ensure we have the proper data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

print("Extracting clinical features...")
# Use the clinical_data obtained directly from the matrix file
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save the clinical data to a CSV file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link clinical and genetic data using the normalized gene data
print("Linking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values in the linked data
print("Handling missing values...")
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Check if trait is biased
print("Checking for bias in trait distribution...")
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Final validation
note = "Dataset contains gene expression data from bronchial brushings from control individuals and patients with asthma after rhinovirus infection in vivo, as described in the study 'Rhinovirus-induced epithelial RIG-I inflammasome suppresses antiviral immunity and promotes inflammation in asthma and COVID-19'."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

print(f"Dataset usability: {is_usable}")

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies due to bias or other issues.")

Normalizing gene symbols...
Gene data shape after normalization: (19847, 136)


Normalized gene data saved to ../../output/preprocess/Alzheimers_Disease/gene_data/GSE122063.csv
Loading the original clinical data...


Extracting clinical features...
Clinical data preview:
{'GSM3454053': [nan, 75.0, 1.0], 'GSM3454054': [nan, 75.0, 1.0], 'GSM3454055': [nan, 75.0, 1.0], 'GSM3454056': [nan, 75.0, 1.0], 'GSM3454057': [nan, 75.0, 1.0], 'GSM3454058': [nan, 75.0, 1.0], 'GSM3454059': [nan, 75.0, 1.0], 'GSM3454060': [nan, 75.0, 1.0], 'GSM3454061': [nan, 90.0, 0.0], 'GSM3454062': [nan, 90.0, 0.0], 'GSM3454063': [nan, 90.0, 0.0], 'GSM3454064': [nan, 90.0, 0.0], 'GSM3454065': [nan, 78.0, 1.0], 'GSM3454066': [nan, 78.0, 1.0], 'GSM3454067': [nan, 78.0, 1.0], 'GSM3454068': [nan, 78.0, 1.0], 'GSM3454069': [nan, 82.0, 0.0], 'GSM3454070': [nan, 82.0, 0.0], 'GSM3454071': [nan, 82.0, 0.0], 'GSM3454072': [nan, 82.0, 0.0], 'GSM3454073': [nan, 96.0, 0.0], 'GSM3454074': [nan, 96.0, 0.0], 'GSM3454075': [nan, 96.0, 0.0], 'GSM3454076': [nan, 96.0, 0.0], 'GSM3454077': [nan, 77.0, 1.0], 'GSM3454078': [nan, 77.0, 1.0], 'GSM3454079': [nan, 77.0, 1.0], 'GSM3454080': [nan, 77.0, 1.0], 'GSM3454081': [nan, 93.0, 0.0], 'GSM3454082': [n

Linked data shape after handling missing values: (100, 19850)
Checking for bias in trait distribution...
For the feature 'Alzheimers_Disease', the least common label is '0.0' with 44 occurrences. This represents 44.00% of the dataset.
The distribution of the feature 'Alzheimers_Disease' in this dataset is fine.

Quartiles for 'Age':
  25%: 77.0
  50% (Median): 81.0
  75%: 83.0
Min: 60.0
Max: 91.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 32 occurrences. This represents 32.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Dataset usability: True


Linked data saved to ../../output/preprocess/Alzheimers_Disease/GSE122063.csv
